
Script to estimate probabilistic power spectral densities for 
one combination of network/station/location/channel/sampling_rate.
(https://docs.obspy.org/tutorial/code_snippets/probabilistic_power_spectral_density.html)

Calculations are based on the routine used by [McNamara2004]:
McNamara, D. E. and Buland, R. P. (2004),
Ambient Noise Levels in the Continental United States,
Bulletin of the Seismological Society of America, 94 (4), 1517-1527.
http://www.bssaonline.org/content/94/4/1517.abstract. 


For information on New High/Low Noise Model see [Peterson1993]:
Peterson, J. (1993),
Observations and Modeling of Seismic Background Noise,
U.S. Geological Survey open-file report 93-322, Albuquerque, N.M.
http://ehp3-earthquake.wr.usgs.gov/regional/asl/pubs/files/ofr93-322.pdf



# Importando as livrarias 

In [1]:
import obspy 
import numpy as np
from obspy import read, Stream,read_inventory
import os
import glob
from obspy import read_inventory
from obspy.signal import PPSD
from obspy.imaging.cm import pqlx

------

This is an example of configuration file, wherein global paths and parameters
related to the analysis of quality of seismological data are defined. 

The parameters are divided in several sections:
- [PATH]                : paths to input/output folders
- [PPSD]                 : parameters to estimate PPSD data
- [CLIENT]               : parameters to request xml file

Before using the scripts, you should at least make sure that the paths in 
section [paths] are consistent with your own files and folders organization. 

-------

# PATH



In [2]:
#-------
#INPUTs

# directory of raw files
DIR_DATA = '/home/dIOGOLOC/dados_posdoc/ON_MAR/dados_NANOMETRICS_test/Projeto6Volumes/9822-9283/2023/'

# XML File
XML_FILE = '/home/dIOGOLOC/dados_posdoc/ON_MAR/dados_NANOMETRICS_test/Projeto6Volumes/9822-9283/responses/ON.S2283_20230920T181425Z.xml'

#-------
#OUTPUTs

#Directory to save Figures
OUTPUT_FIGURE_DIR = '/home/dIOGOLOC/dados_posdoc/ON_MAR/dados_NANOMETRICS_test/OUTPUT/FIGURES/'

#Directory to save PSD
OUTPUT_PSD_DIR = '/home/dIOGOLOC/dados_posdoc/ON_MAR/dados_NANOMETRICS_test/OUTPUT/PPSD/'

# PPSD


In [3]:
# Initial date
INITIAL_DATE = 2023,9,1

# Final date
FINAL_DATE = 2023,9,31

# Restricts the data that is included in the stack by time of day and weekday:
    # Monday is 1, Sunday is 7, -1 for any day of week. 
    
    # For example, using time_of_weekday=[(-1, 22, 24)] 
    # only individual spectra that have a starttime in between 10pm and 12am are used in the stack for all days of week
    # time_of_weekday=[(TIME_OF_WEEKDAY_DAY, TIME_OF_WEEKDAY_START_HOUR, TIME_OF_WEEKDAY_FINAL_HOUR)])
    
TIME_OF_WEEKDAY_DAY = -1
TIME_OF_WEEKDAY_START_HOUR = 0
TIME_OF_WEEKDAY_FINAL_HOUR = 24

#Maximum and minimum amplitude of the PSD. 
AMP_PSD_MIN = -155
AMP_PSD_MAX = -80

# Functions

In [4]:
def calc_save_PPSD(dataMSEED,file_xml):
    '''
    
    Function to estimate and save probabilistic power spectral densities 
    for one combination of network/station/location/channel/sampling_rate.
        
    Args:
        dataMSEED (miniseed): daily file record.
        file_xml (StationXML): metadata that describes the data collected.
        
    Returns:
        binary file: Probabilistic power spectral densities of daily miniseed file.
    
    '''
    
    # Importing XML file
    inv = read_inventory(file_xml)
    
    # Reading MSEED file

    st = read(dataMSEED)

    time_data = st[0].stats.starttime
    time_data_year = '{:04}'.format(time_data.year)
    time_data_julday = '{:03}'.format(time_data.julday)
    time_data_hour = '{:02}'.format(time_data.hour)
    time_data_minute = '{:02}'.format(time_data.minute)

    sta_name = st[0].stats.station
    sta_channel = st[0].stats.channel
        
    print('Calculating PPSD: station: '+sta_name+' / channel: '+sta_channel)

    ppsd = PPSD(st[0].stats, metadata=inv)
    ppsd.add(st)
        
    os.makedirs(OUTPUT_PSD_DIR+time_data_year+'/'+sta_name+'/'+sta_channel+'.PPSD'+'/',exist_ok=True)
    ppsd.save_npz(OUTPUT_PSD_DIR+time_data_year+'/'+sta_name+'/'+sta_channel+'.PPSD'+'/'+sta_name+'..'+sta_channel+'.PPSD'+'.'+time_data_year+'.'+time_data_julday+'.npz')

In [6]:
def plot_PPSD_TOTAL_data(dataPPSD):
    '''
    
    Function to plot and save probabilistic power spectral densities images.
        
    Args:
        dataPPSD (npz): daily binary file.
        
    Returns:
        image: PDF file.
    
    '''
    
    os.chdir(dataPPSD)
    files = sorted(glob.glob('*.npz'))
    ppsd = PPSD.load_npz(files[0],allow_pickle=True)

    [ppsd.add_npz(i,allow_pickle=True) for i in files[1:]]
    os.makedirs(OUTPUT_FIGURE_DIR+'TOTAL/'+ppsd.station+'/',exist_ok=True)
    ppsd.plot_temporal([0.1, 1, 10],filename=OUTPUT_FIGURE_DIR+'TOTAL/'+ppsd.station+'/'+ppsd.station+'.'+ppsd.channel+'.temporal.pdf')
    ppsd.plot_spectrogram(filename=OUTPUT_FIGURE_DIR+'TOTAL/'+ppsd.station+'/'+ppsd.station+'.'+ppsd.channel+'.spectrogram.pdf')
    ppsd.plot(cmap=pqlx,filename=OUTPUT_FIGURE_DIR+'TOTAL/'+ppsd.station+'/'+ppsd.station+'.'+ppsd.channel+'.pqlx.pdf')

# PPSD: estimation and visualization

In [7]:
# ========================================
# Importing data from raw files directory 
# ========================================

data_lista = []

for root, dirs, files in os.walk(DIR_DATA):
    for name in files:
        data_lista.append(os.path.join(root, name))

data_lista = sorted(data_lista)

In [8]:
# ==============
#  Get PPSD Data 
# ==============

for i in data_lista:
    calc_save_PPSD(i,XML_FILE)

Calculating PPSD: station: S2283 / channel: HHE


/home/dIOGOLOC/programs/anaconda3/lib/python3.10/site-packages/obspy/signal/spectral_estimation.py:968: UserWarning: Error getting response from provided metadata:
Exception: No matching response information found.
Skipping time segment(s).
  warnings.warn(msg)


Calculating PPSD: station: S2283 / channel: HHE
Calculating PPSD: station: S2283 / channel: HHN


/home/dIOGOLOC/programs/anaconda3/lib/python3.10/site-packages/obspy/signal/spectral_estimation.py:968: UserWarning: Error getting response from provided metadata:
Exception: No matching response information found.
Skipping time segment(s).
  warnings.warn(msg)


Calculating PPSD: station: S2283 / channel: HHN
Calculating PPSD: station: S2283 / channel: HHZ


/home/dIOGOLOC/programs/anaconda3/lib/python3.10/site-packages/obspy/signal/spectral_estimation.py:968: UserWarning: Error getting response from provided metadata:
Exception: No matching response information found.
Skipping time segment(s).
  warnings.warn(msg)


Calculating PPSD: station: S2283 / channel: HHZ


In [9]:
# ===========================
# Finding stations PPSD data
# ===========================

datafile_lst = [] 
for root, dirs, files in os.walk(OUTPUT_PSD_DIR):
    for directories in dirs:
        datafile_name = os.path.join(root, directories)
        if '.PPSD' in datafile_name:
            datafile_lst.append(datafile_name+'/')
datafile_lstS = sorted(datafile_lst)

In [10]:
datafile_lstS

['/home/dIOGOLOC/dados_posdoc/ON_MAR/dados_NANOMETRICS_test/OUTPUT/PPSD/2023/S2283/HHE.PPSD/',
 '/home/dIOGOLOC/dados_posdoc/ON_MAR/dados_NANOMETRICS_test/OUTPUT/PPSD/2023/S2283/HHN.PPSD/',
 '/home/dIOGOLOC/dados_posdoc/ON_MAR/dados_NANOMETRICS_test/OUTPUT/PPSD/2023/S2283/HHZ.PPSD/']

In [11]:
# ================
#  plot PPSD Data 
# ================

for i,j in enumerate(datafile_lstS):
    plot_PPSD_TOTAL_data(j)

/home/dIOGOLOC/programs/anaconda3/lib/python3.10/site-packages/obspy/signal/spectral_estimation.py:1042: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  selected = np.ones(len(times_all), dtype=np.bool)


AttributeError: module 'numpy' has no attribute 'bool'.
`np.bool` was a deprecated alias for the builtin `bool`. To avoid this error in existing code, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations